In [ ]:
#cleaning description text in training set
import pandas as pd
full_df = pd.read_csv('train-v7.csv')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence=sentence.lower()
    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
    sentence=re.sub('_',' ', sentence) #removes punctuations
    sentence=re.sub('\d+',' ', sentence) #removes digits
    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
    cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    
#selecting only nouns and adjectives
    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
    cleaned=cleaned.strip()
    return cleaned
	  
full_df['cleaned'] = full_df['description'].apply(lambda x: cleaning_text(x))
trimDF_train = full_df[['aptID','cleaned','interest_level']]

In [ ]:
text_train = trimDF['cleaned']
import numpy as np
text_train = text_train.replace(np.nan, '')

In [ ]:
from time import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [ ]:
n_features = 1000
n_topics = 10
n_top_words = 20


tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(text_train)
TF = TfidfTransformer()
tf_idf = TF.fit_transform(tf)
tf_idf = tf_idf.toarray() #convert sparse matrix to array
#tf_idf.toarray()[:100, :100]

In [ ]:
tf_idf = pd.DataFrame(tf_idf)
trimDF_train = pd.concat([trimDF_train.drop(['cleaned','Unnamed: 0'], axis=1), tf_idf.add_prefix('tf_idf')], axis=1)

In [ ]:
#logistic regression model predicting based on df_idf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
#nrow = trimDF_tfidf.shape[0]

train, test = train_test_split(trimDF_tfidf, train_size=0.5, random_state=1)
Train = pd.DataFrame(train, columns=train.columns)
Test = pd.DataFrame(test, columns=test.columns)

x = Train[Train.columns.values[2:1002]]
y = Train['interest_level']
x2 = Test[Test.columns.values[2:1002]]
y2 = Train['interest_level']

x_total = trimDF_tfidf[trimDF_tfidf.columns.values[2:1002]]
y_total = trimDF_tfidf['interest_level']
model_total = LogisticRegression()
model_total.fit(x_total,y_total)
#use first half data train model and predict the second half
model = LogisticRegression()
model.fit(x, y)
model.predict(x2)
new_col_Test = model.predict_proba(x2)
# save the model to disk
filename = 'description_prediction_model_log.sav'
pickle.dump(model, open(filename, 'wb'))

model_2 = LogisticRegression()
model_2.fit(x2, y2)
model_2.predict(x)
new_col_Train = model_2.predict_proba(x)
# save the model to disk
filename = 'description_prediction_model_2_log.sav'
pickle.dump(model_2, open(filename, 'wb'))


#type(new_col_1)
col_Test = ['high','low','medium']
new_col_Test = pd.DataFrame(new_col_Test, columns = col_Test)

col_Train = ['high','low','medium']
new_col_Train = pd.DataFrame(new_col_Train, columns = col_Train)
#model.predict(x[:100])

Train_tfidf = pd.concat([Train.ix[:,[0]].reset_index(drop=True),new_col_Train.ix[:,[0,2]]],axis=1)
Test_tfidf = pd.concat([Test.ix[:,[0]].reset_index(drop=True),new_col_Test.ix[:,[0,2]]],axis=1)

trimDF_train_prob = Train_tfidf.append(Test_tfidf, ignore_index=True)

#fit model with whole training set
x_total = trimDF_tfidf[trimDF_tfidf.columns.values[2:1002]]
y_total = trimDF_tfidf['interest_level']
model_total = LogisticRegression()
model_total.fit(x_total,y_total)
# save the model to disk
filename = 'description_prediction_model_total_log.sav'
pickle.dump(model_total, open(filename, 'wb'))

In [ ]:
trimDF_train_prob.to_csv('description_prob_train_logit_v4.csv', index =False)

In [ ]:
#cleaning description text in test set
import pandas as pd
full_df = pd.read_csv('test-v7.csv')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence=sentence.lower()
    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
    sentence=re.sub('_',' ', sentence) #removes punctuations
    sentence=re.sub('\d+',' ', sentence) #removes digits
    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
    cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    
#selecting only nouns and adjectives
    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
    cleaned=cleaned.strip()
    return cleaned
	  
full_df['cleaned'] = full_df['description'].apply(lambda x: cleaning_text(x))
trimDF_test = full_df[['aptID','cleaned']]

In [ ]:
text = trimDF_test['cleaned']

In [ ]:
#delete all nan in text
import numpy as np
text = text.replace(np.nan, '')

In [ ]:
from time import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [ ]:
#computing the term frequency for the selected words:
n_features = 1000
n_topics = 10
n_top_words = 20


tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(text)
TF = TfidfTransformer()
#compute tf_idf
tf_idf = TF.fit_transform(tf)
tf_idf = tf_idf.toarray() #convert sparse matrix to array
#tf_idf.toarray()[:100, :100]
tf_idf = pd.DataFrame(tf_idf)

In [ ]:
trimDF_test_tfidf = pd.concat([trimDF_test.drop(['cleaned','Unnamed: 0'], axis=1), tf_idf.add_prefix('tf_idf')], axis=1)

In [ ]:
import pickle
X = trimDF_tfidf[trimDF_tfidf.columns.values[1:1002]]
# load the model from disk
loaded_model = pickle.load(open('description_prediction_model_total_log.sav', 'rb'))
result = loaded_model.predict_proba(X)

In [ ]:
col_test = ['high','low','medium']
result = pd.DataFrame(result, columns = col_test)

colsToDrop = ['low']
result = result.drop(colsToDrop, axis=1)

test_prob = pd.concat([trimDF_tfidf.ix[:,[0]].reset_index(drop=True),result.ix[:,[0,1]]],axis=1)

test_prob.to_csv('description_prob_test_logit_v4.csv', index =False)